# Advanced Financial Market Analysis (Static): Navigating Stock Trends with Bollinger Bands and VWAP

> ## Overview

> This notebook embarks on an advanced journey into financial market analysis, employing statistical tools like Bollinger Bands and the Volume-Weighted Average Price (VWAP) to scrutinize stock data. Aimed at traders and analysts, it offers a meticulous exploration of stock behavior over 1-minute and 20-minute intervals, providing a robust framework for understanding market dynamics. Through calculated metrics, insightful visualizations, and a proactive alert system, this guide serves as an invaluable resource for crafting data-driven trading strategies.

In [1]:
%%capture -- no-display
%pip install pathway

### Import Libraries

The first block of code imports necessary libraries for the script. 

- `datetime`: Provides classes for manipulating dates and times.
- `requests`: Allows sending HTTP requests to fetch data from the web.
- `pandas`: Offers data structures and operations for manipulating numerical tables and time series.
- `bokeh.models` and `bokeh.plotting`: Used for creating interactive plots and visualizations.
- `panel`: Provides tools for creating web apps and dashboards.
- `pathway` (aliased as `pw`): Custom library, likely for data manipulation or pathway analysis.
- `polygon`: Provides a RESTClient, possibly for accessing financial data.


In [2]:
import datetime
import requests
import pandas as pd

import bokeh.models
import bokeh.plotting
import panel

import pathway as pw

from polygon import RESTClient


> ### Read and Display CSV Data
This block reads data from a CSV file named ticker.csv into a pandas DataFrame. The index_col=0 parameter sets the first column as the index of the DataFrame. It then displays the DataFrame.

In [3]:
fname = 'ticker.csv'
df = pd.read_csv(fname, index_col=0)
df

,open,high,low,close,volume,vwap,t,transactions,otc
ticker,,,,,,,,,
AAPL,189.60,189.60,189.20,189.30,473.0,189.4087,1701075600000,23,NaN
AAPL,189.20,189.20,189.20,189.20,418.0,189.1945,1701075610000,7,NaN
AAPL,189.25,189.25,189.25,189.25,301.0,189.2500,1701075764000,2,NaN
AAPL,189.25,189.25,189.25,189.25,105.0,189.2500,1701075784000,2,NaN
AAPL,189.27,189.27,189.27,189.27,250.0,189.2758,1701076340000,3,NaN
...,...,...,...,...,...,...,...,...,...
AAPL,190.15,190.15,190.10,190.10,422.0,190.1251,1701133062000,7,NaN
AAPL,190.09,190.09,190.09,190.09,128.0,190.0900,1701133138000,1,NaN
AAPL,190.03,190.03,190.03,190.03,152.0,190.0300,1701133176000,1,NaN


> ### Data Schema and Reading with Custom Parameters

Defines a schema from the CSV file using a custom library (pathway) and reads the data again with this schema. The data is read in a 'static' mode, which could mean without expecting updates or changes.

In [4]:
schema = pw.schema_from_csv(fname)
data = pw.io.csv.read(fname, schema=schema, mode='static')
data

[2024-03-07T16:31:59]:INFO:Preparing Pathway computation
[2024-03-07T16:31:59]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:31:59]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:31:59]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'3ee75f7d-1bc1-4579-a4d3-fd1a8760420c': {'version…

> ### Data Transformation
Converts the timestamps in column 't' from milliseconds to UTC datetime format. This transformation is important for time series analysis, ensuring that time data is correctly interpreted.

In [5]:
data = data.with_columns(t=data.t.dt.utc_from_timestamp(unit='ms'))
data

[2024-03-07T16:32:00]:INFO:Preparing Pathway computation
[2024-03-07T16:32:00]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:00]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:00]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'0ad3b49c-fb76-45d5-a8ba-58c4309e99fd': {'version…

> ### Visualization with Bokeh
Defines a function vwap_history that creates a Bokeh figure to visualize the volume-weighted average price (VWAP) history. The figure uses datetime on the x-axis and VWAP on the y-axis. It then applies this visualization function to the data, sorting by column 't'.

In [6]:
def vwap_history(src):
    fig = bokeh.plotting.figure(
        height=400, width=800,
        title='Volume-weighted average price',
        x_axis_type='datetime'
    )
    fig.line('t', 'vwap', source=src)
    return fig

In [7]:
data.plot(vwap_history, sorting_col='t')

[2024-03-07T16:32:00]:INFO:Preparing Pathway computation


[2024-03-07T16:32:00]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:00]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:00]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'7d25000e-4233-4b1e-8733-eee794765ebd': {'version…

## Sliding Window Analysis for Financial Data

This code snippet demonstrates a sophisticated analysis on financial data by calculating various statistics over 20-minute windows that slide one minute at a time. It is tailored to process time series data for each ticker symbol, focusing on generating detailed minute-by-minute statistics.

### Key Operations

1. **Sliding Window Configuration**:
   - Utilizes a sliding window approach where each window spans 20 minutes and moves forward by one minute for each calculation.
   - This method is ideal for analyzing time series data, providing insights into short-term trends and metrics that evolve over time.

2. **Data Reduction**:
   - Aggregates data within each window to calculate total volume, transaction total, and a squared version of the transaction total.
   - These aggregated metrics serve as the basis for more complex calculations such as VWAP and standard deviation.

3. **Volume-Weighted Average Price (VWAP)**:
   - Computes VWAP for each window, which represents the average price a security has traded at, weighted by volume.
   - VWAP is a crucial metric for assessing the market's trend and the efficiency of trade executions.

4. **VWAP Standard Deviation**:
   - Calculates the standard deviation of VWAP, offering insights into the volatility of the price over the window.
   - This metric helps in understanding the distribution and variability of trade prices around the VWAP.

5. **Bollinger Bands**:
   - Generates Bollinger Bands (upper and lower) using the VWAP and its standard deviation.
   - Bollinger Bands are widely used in stock market analysis to identify overbought or oversold conditions.


In [8]:
minute_20_stats = (
    data.windowby(
        pw.this.t,
        window=pw.temporal.sliding(
            hop=datetime.timedelta(minutes=1),
            duration=datetime.timedelta(minutes=20)
        ),
        instance=pw.this.ticker
    )
    .reduce(
        ticker=pw.this._pw_instance,
        t=pw.this._pw_window_end,
        volume=pw.reducers.sum(pw.this.volume),
        transact_total=pw.reducers.sum(pw.this.volume * pw.this.vwap),
        transact_total2=pw.reducers.sum(pw.this.volume * pw.this.vwap**2)
    )
    .with_columns(vwap=pw.this.transact_total / pw.this.volume)
    .with_columns(vwstd=(pw.this.transact_total2 / pw.this.volume - pw.this.vwap**2)**0.5)
    .with_columns(
        bollinger_upper=pw.this.vwap + 2 * pw.this.vwstd,
        bollinger_lower=pw.this.vwap - 2 * pw.this.vwstd
    )
)
minute_20_stats

[2024-03-07T16:32:02]:INFO:Preparing Pathway computation
[2024-03-07T16:32:02]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:02]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:02]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'280a3902-1db2-4b9e-ab2b-6e4b5d1ce8d9': {'version…

## Simplified Explanation of Calculating 1-Minute Window Statistics

In this code snippet, we're focusing on analyzing financial data within 1-minute intervals for each stock ticker. The process involves segmenting the data into 1-minute blocks (or windows), then calculating key statistics within each block. Here's a breakdown of each step and what it aims to achieve:

### Step 1: Creating 1-Minute Windows

- **`windowby` Method**: We organize the data into 1-minute windows using a "tumbling" window method. Unlike sliding windows, tumbling windows do not overlap; each segment is distinct, covering exactly one minute of trading time.
- **`instance` Parameter**: This ensures that calculations are done separately for each stock ticker, allowing for ticker-specific insights.

### Step 2: Summarizing Data Within Each Window

- **`reduce` Function**: This part aggregates the trading data for each 1-minute window. Two key metrics are calculated:
  - **`volume`**: The total number of shares traded in that minute. This tells us how active the market was.
  - **`transact_total`**: The total dollar amount traded, found by multiplying each trade's volume by the trade price (VWAP) and then summing these. It indicates the total value of transactions.

### Step 3: Calculating the Volume-Weighted Average Price (VWAP)

- **`with_columns` Method**: After summing up the total transaction value and volume, we calculate the VWAP for each minute.
  - **`vwap`**: This is the ratio of `transact_total` to `volume`, providing the average price at which shares were traded, weighted by the number of shares. VWAP is crucial for understanding the market's price trend within the minute.

### In Simple Terms:

1. **Segment the data** into non-overlapping, 1-minute blocks for each ticker symbol.
2. **Summarize trading activity** by calculating the total volume and value of trades within each minute.
3. **Determine the average trading price** (VWAP) for each minute, offering a snapshot of price trends and market activity.

This method allows traders and analysts to closely monitor market movements and trading patterns on a minute-by-minute basis, providing insights into short-term market dynamics.


In [9]:
minute_1_stats = (
    data.windowby(
        pw.this.t,
        window=pw.temporal.tumbling(datetime.timedelta(minutes=1)),
        instance=pw.this.ticker
    )
    .reduce(
        ticker=pw.this._pw_instance,
        t=pw.this._pw_window_end,
        volume=pw.reducers.sum(pw.this.volume),
        transact_total=pw.reducers.sum(pw.this.volume * pw.this.vwap)
    )
    .with_columns(vwap=pw.this.transact_total / pw.this.volume)
)
minute_1_stats

[2024-03-07T16:32:12]:INFO:Preparing Pathway computation
[2024-03-07T16:32:12]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:12]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:12]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'66d3da9b-b679-4ba7-a801-e5c9c435fe8e': {'version…

## Combining 1-Minute and 20-Minute Statistics for Market Alerts

This code snippet demonstrates how to merge two sets of financial statistics—1-minute and 20-minute interval calculations—into a comprehensive dataset. It then uses this combined data to identify potential trading alerts based on predefined criteria. Let's break down the process:

### Step 1: Joining Datasets

- **`join` Operation**: The 1-minute and 20-minute statistics are merged based on matching timestamps (`t`) and ticker symbols. This creates a dataset that includes both the short-term and longer-term analytics for each stock.

### Step 2: Selecting Relevant Metrics

- **`select` Method**: After merging, we retain all metrics from the 1-minute statistics and also include the Bollinger Bands (`bollinger_lower` and `bollinger_upper`) from the 20-minute stats. Bollinger Bands are used to identify price volatility and potential trading opportunities.

### Step 3: Identifying Trading Alerts

- **`is_alert` Column**: A new column is calculated to flag potential trading alerts. An alert is triggered if:
  - The trading volume exceeds 10,000 shares, indicating significant market activity.
  - The VWAP is either above the upper Bollinger Band (suggesting the stock may be overvalued) or below the lower Bollinger Band (indicating the stock may be undervalued).

### Step 4: Determining Trading Action

- **`action` Column**: Based on the alert condition, a suggested trading action is determined:
  - If an alert is triggered and the VWAP is above the upper Bollinger Band, it suggests selling the stock ("sell").
  - Conversely, if the VWAP is below the lower Bollinger Band, buying is recommended ("buy").
  - If no alert condition is met, the action is to "hold" (represented as "hodl", a term popularly used in the cryptocurrency community to denote holding onto assets rather than selling).

### In Simple Terms:

1. **Merge the short-term and long-term analytics** for a comprehensive view of each stock.
2. **Utilize Bollinger Bands** from the longer-term stats to assess market volatility and potential overbought or oversold conditions.
3. **Flag significant trading opportunities** based on volume and price conditions relative to the Bollinger Bands.
4. **Suggest actions** (buy, sell, hold) based on whether specific alert criteria are met.

This approach allows traders to systematically identify and act on potential trading opportunities by leveraging both short-term movements and longer-term trends.


In [10]:
joint_stats = (
    minute_1_stats.join(
        minute_20_stats, pw.left.t == pw.right.t, 
        pw.left.ticker == pw.right.ticker
    )
    .select(
        *pw.left,
        bollinger_lower = pw.right.bollinger_lower,
        bollinger_upper=pw.right.bollinger_upper
    )
    .with_columns(
        is_alert=(
            (pw.this.volume > 10000)
            & (
                (pw.this.vwap > pw.this.bollinger_upper)
                | (pw.this.vwap < pw.this.bollinger_lower)
            )
        )
    )
    .with_columns(
        action=pw.if_else(
            pw.this.is_alert,
            pw.if_else(pw.this.vwap > pw.this.bollinger_upper, 'sell', 'buy'),
            'hodl'
        )
    )
)
joint_stats

[2024-03-07T16:32:14]:INFO:Preparing Pathway computation
[2024-03-07T16:32:14]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:14]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:14]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'e1dcc526-453c-4ef7-8fbe-05809434e2bf': {'version…

## Filtering for Trading Alerts

This code snippet focuses on extracting specific trading alerts from the combined dataset (`joint_stats`) that we previously created by merging 1-minute and 20-minute financial statistics. The goal is to isolate instances that meet our criteria for significant trading opportunities, based on volume and VWAP's relation to Bollinger Bands. Here's how it's done:

### Step 1: Filtering for Alerts

- **`filter` Method**: We apply a filter to the `joint_stats` dataset to keep only those records where an alert has been triggered. An alert is considered triggered if the trading volume is above 10,000 and the VWAP is significantly higher or lower than expected, indicated by its position relative to the Bollinger Bands.

### Step 2: Selecting Key Information

- **`select` Method**: For each record that triggers an alert, we extract only the most relevant information:
  - **`ticker`**: The stock symbol for which the alert is generated.
  - **`t`**: The timestamp indicating when the alert condition was met.
  - **`vwap`**: The Volume-Weighted Average Price at the time of the alert, which signals whether the stock is potentially overvalued or undervalued.
  - **`action`**: The suggested trading action (buy, sell, or hold) based on the alert's criteria.

### The Result: A Streamlined Dataset of Alerts

The `alerts` dataset now contains a concise list of moments when specific stocks present potential trading opportunities. For each alert, it provides the ticker symbol, the precise time of the alert, the current VWAP (giving context to the price level), and a suggested action to take. 

### In Simple Terms:

1. **Isolate important moments** where the trading volume and price movement indicate a significant opportunity or risk.
2. **Extract critical details** about these moments, including which stock is involved, when the opportunity arose, what the price was, and what action might be advisable.
3. **Prepare a focused dataset** of alerts that can be used to make informed trading decisions quickly.

This streamlined process enables traders and analysts to rapidly identify and react to key trading signals, potentially enhancing their trading strategy's effectiveness.


In [11]:
alerts = (
    joint_stats
    .filter(pw.this.is_alert)
    .select(pw.this.ticker, pw.this.t, pw.this.vwap, pw.this.action)
)
alerts

[2024-03-07T16:32:27]:INFO:Preparing Pathway computation
[2024-03-07T16:32:27]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:27]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:27]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'00aa4c4d-d315-4f80-a605-34a2ade1bc84': {'version…

## Visualizing Financial Data with Bollinger Bands and Trading Actions

The `stats_plotter` function is designed to create an informative and interactive visualization of financial data, specifically focusing on the use of Bollinger Bands and trading actions (buy, sell, hold) over a 20-minute window. Here's a detailed breakdown of the function and its components:

### Function Definition: `stats_plotter(src)`

- **Input Parameter**: `src`, which should be a data source containing the necessary financial metrics, including timestamps (`t`), Volume-Weighted Average Price (VWAP), Bollinger Bands (`bollinger_lower`, `bollinger_upper`), and trading actions.

### Key Components of the Visualization:

1. **Action Color Mapping**:
   - Initializes a color map for different actions, using green for "buy", red for "sell", and transparent for "hold". This visual cue helps users quickly identify recommended actions.

2. **Bokeh Figure Configuration**:
   - Creates a Bokeh figure with a specific height, width, and title, and sets the x-axis type to 'datetime' for appropriate handling of time series data.

3. **VWAP Line Plot**:
   - Draws a line to represent the VWAP over time, offering insights into the average price movement within the last minute.

4. **Bollinger Bands Visualization**:
   - Plots the Bollinger Bands with semi-transparent lines and fills the area between them in gray (also semi-transparent). This graphical representation helps visualize stock volatility and the expected trading range.

5. **Trading Action Scatter Plot**:
   - Adds scatter points on top of the VWAP line, colored according to the predefined actions, to pinpoint specific times where buying or selling is suggested based on the analysis.

### Visualization Goal:

The function culminates in a comprehensive plot that combines time series analysis with actionable trading insights. It allows traders and analysts to visually assess market trends, volatility, and potential trading opportunities based on short-term price movements and Bollinger Band indicators.

### Implementation:

```python
joint_stats.plot(stats_plotter, sorting_col='t')


In [12]:
def stats_plotter(src):
    actions = ['buy', 'sell', 'hodl']
    color_map = bokeh.models.CategoricalColorMapper(
        factors=actions,
        palette=('#00ff00', '#ff0000', '#00000000')
    )

    fig = bokeh.plotting.figure(
        height=500, width=800,
        title='20 minutes Bollinger bands with last 1 minute average',
        x_axis_type='datetime'
    )

    fig.line('t', 'vwap', source=src)

    fig.line('t', 'bollinger_lower', source=src, line_alpha=0.3)
    fig.line('t', 'bollinger_upper', source=src, line_alpha=0.3)
    fig.varea(
        x='t',
        y1='bollinger_lower',
        y2='bollinger_upper',
        fill_alpha=0.3,
        fill_color='gray',
        source=src,
    )

    fig.scatter(
        't', 'vwap', size=10, marker='circle', color={'field': 'action', 'transform': color_map},
        source=src
    )

    return fig

joint_stats.plot(stats_plotter, sorting_col='t')

[2024-03-07T16:32:38]:INFO:Preparing Pathway computation
[2024-03-07T16:32:38]:INFO:CsvFilesystemReader-0: 0 entries (1 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:38]:INFO:CsvFilesystemReader-0: 18329 entries (2 minibatch(es)) have been sent to the engine
[2024-03-07T16:32:38]:WARNING:CsvFilesystemReader-0: Closing the data source


BokehModel(combine_events=True, render_bundle={'docs_json': {'67a32899-3f4c-4d53-a710-3b428791da97': {'version…

## Summary

### Key Highlights:

- **Data Preparation**: Initiates the process with a comprehensive setup for reading and organizing financial time series data, making it amenable to detailed analysis.
- **Statistical Analysis**: At the heart of this notebook is the utilization of Bollinger Bands and VWAP. These tools help dissect market trends and volatility, offering a nuanced understanding that is critical for successful market navigation.
- **Alert System**: Introduces an innovative alert mechanism that flags significant trading opportunities. This system is based on meticulously analyzed volume and price movements, ensuring traders and analysts can act swiftly on potential gains.
- **Visualization**: Leverages the power of Bokeh plots to present data in a visually engaging manner. This not only enhances the comprehension of market trends but also aids in making informed trading decisions.

## Conclusion
> The notebook presents a holistic approach to financial market analysis, blending sophisticated statistical techniques with practical tools for real-time market evaluation. It is designed to empower traders and analysts with the knowledge and tools necessary to make informed decisions, backed by data-driven insights into market trends and movements.